<a href="https://colab.research.google.com/github/dho619/TCC_Pos_IA/blob/main/TratamentoDosDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello Word!");

Hello Word!
